# PROUNI

## Definição de Variáveis e Imports

In [ ]:
#!pip install unidecode
from requests import get
from pandas import DataFrame, to_numeric, read_csv, concat, to_datetime, Series, read_excel
from unidecode import unidecode
from bs4 import BeautifulSoup
from os import path, system, remove, makedirs
from glob import glob
from re import sub, search



data_dir_prouni = '/content/downloads/prouni'
data_dir_ibge = '/content/downloads/ibge'

resultados_dir = '/content/resultados'

limit = 2000

url_prouni = f'https://dadosabertos.mec.gov.br'
url_ibge = f'https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/23701-divisao-territorial-brasileira.html'

title = '/prouni'


##  Função de Extração dos links PROUNI

In [ ]:

class Prouni:
  def __init__(self, url, start_year, title):
    self.url = url
    self.start_year = start_year
    self.title = title

  def get_links(self):

    response = get(f'{self.url}{self.title}')
    response = BeautifulSoup(response.text, 'html.parser')
    pages = [self.title] + list(set([f"{elem['href']}" for elem in response.find_all(
        "a", {"class": "hasTooltip pagenav"}, href=True) if elem['href']]))

    finaldata = DataFrame()

    for page in pages:
      url_name= f"{self.url}{page}"
      response = get(url_name)
      response = BeautifulSoup(response.text, 'html.parser')

      data = DataFrame(
                      {"url": [f"{elem['href']}"
                               for elem in response.find_all("a", href=True)
                              if elem["href"].endswith(('.csv', '.zip'))]}
                  ).assign(
                      arquivo=lambda x: x.url.str.replace(".*/", "", regex=True),
                      ano=lambda x: to_numeric(x.arquivo.str.findall("[0-9]+").str.join(""), errors="coerce"
                      ).astype(int),
                   )

      finaldata = concat([finaldata, data], ignore_index=False)
    finaldata = finaldata[~finaldata['url'].str.contains('v3')]

    finaldata = finaldata.copy().query(f"ano >= {self.start_year}")
    return finaldata


  def download(self, data_dir):

    urls = self.get_links()

    if not path.exists(data_dir):
      print("Criando pasta")
      makedirs(data_dir)


    for doc in urls.to_dict('records'):
      print(doc)
      file = doc["arquivo"]
      urlname = doc["url"]
      year = doc["ano"]
      destfile = (f'{data_dir}/{file}')
      if destfile in glob(f"{data_dir}/*"):
        print("Arquivo já existe")
        continue

      os_cmd = (
            f"wget --limit-rate {limit}k "
            f"--no-check-certificate -c {urlname} -O {destfile}")
      system(os_cmd)
      if ".zip" in file.lower():
        print("Uncompress ZIP file")
        system(f"7z e {destfile} -O{data_dir}")
        print("Remove ZIP file")
        remove(destfile)
    def transform(self, data_dir, resultados_dir):

      data = data.read_csv(f'{data_dir}/*.csv')

      return data.head()



## Função de Extração links ibge

In [ ]:

class IBGE:
  def __init__(self):
    self.url = None

  def get_links(self):

    response = get(f'{self.url}')
    response = BeautifulSoup(response.text, 'html.parser')

    data = DataFrame(
                      {"url": [f"{elem['href']}"
                               for elem in response.find_all("a", href=True)
                              if elem["href"].endswith(('.csv', '.zip'))]}
                  ).assign(
                      arquivo=lambda x: x.url.str.replace(".*/", "", regex=True),
                      ano=lambda x: to_numeric(x.arquivo.str.findall("[0-9]+").str.join(""), errors="coerce"
                      ).astype(int),
                   )



    return data

  def download(self,url, data_dir, start_year):
    self.url = url
    urls = self.get_links()

    data = urls.query(f"ano >= {start_year}")
    print(data)

    if not path.exists(data_dir):
      print("Criando pasta")
      makedirs(data_dir)


    for doc in data.to_dict('records'):
      print(doc)
      file = doc["arquivo"]
      urlname = doc["url"]
      year = doc["ano"]
      destfile = (f'{data_dir}/{file}')

      if destfile in glob(f"{data_dir}/*"):
        print("Arquivo já existe")
        continue

      os_cmd = (
            f"wget --limit-rate {limit}k "
            f"--no-check-certificate -c {urlname} -O {destfile}")
      system(os_cmd)

      if ".zip" in file.lower():
        print("Uncompress ZIP file")
        system(f"7z e {destfile} -O{data_dir}")
        print("Remove ZIP file")
        remove(destfile)



## Usando as funções

In [ ]:
## Baixando Dados do prouni
prouni = Prouni(url_prouni, 2020, title)
urls = prouni.get_links()
prouni.download(data_dir_prouni)


{'url': 'https://dadosabertos.mec.gov.br/images/conteudo/prouni/2020/ProuniRelatorioDadosAbertos2020.csv', 'arquivo': 'ProuniRelatorioDadosAbertos2020.csv', 'ano': 2020}
Arquivo já existe


In [ ]:
urls

,url,arquivo,ano
0,https://dadosabertos.mec.gov.br/images/conteud...,ProuniRelatorioDadosAbertos2020.csv,2020


In [ ]:
#finaldata = DataFrame()
#def read_file(file):
#  file = read_csv(file, sep=';', encoding='utf8')
#  if 'ï»¿' in file:
#    return read_csv(file, sep=';', encoding='latin1')
#  else:
#    return file


files = glob(f"{data_dir_prouni}*/*.csv")
for file in files:
  prouni = read_csv(file, sep=';', encoding='latin1')



In [ ]:
prouni

,ANO_CONCESSAO_BOLSA,CODIGO_EMEC_IES_BOLSA,NOME_IES_BOLSA,MUNICIPIO,CAMPUS,TIPO_BOLSA,MODALIDADE_ENSINO_BOLSA,NOME_CURSO_BOLSA,NOME_TURNO_CURSO_BOLSA,CPF_BENEFICIARIO,SEXO_BENEFICIARIO,RACA_BENEFICIARIO,DATA_NASCIMENTO,BENEFICIARIO_DEFICIENTE_FISICO,REGIAO_BENEFICIARIO,UF_BENEFICIARIO,MUNICIPIO_BENEFICIARIO
0,2020,322,UNIVERSIDADE PAULISTA,IPATINGA,IPATINGA,INTEGRAL,EAD,PEDAGOGIA,CURSO A DISTÂNCIA,991.XXX.XXX-91,F,Parda,11/08/1973,N,SUDESTE,MG,GOVERNADOR VALADARES
1,2020,163,UNIVERSIDADE ESTÁCIO DE SÁ,FORTALEZA,EAD VIA CORPVS - CE,INTEGRAL,EAD,MARKETING,CURSO A DISTÂNCIA,067.XXX.XXX-01,M,Parda,13/05/1987,N,NORDESTE,CE,FORTALEZA
2,2020,17670,FACULDADE DE QUIXERAMOBIM,QUIXERAMOBIM,FACULDADE DE QUIXERAMOBIM - UNIQ,INTEGRAL,PRESENCIAL,FARMÁCIA,NOTURNO,623.XXX.XXX-27,M,Parda,23/07/2001,N,NORDESTE,CE,MOMBACA
3,2020,203,UNIVERSIDADE SÃO JUDAS TADEU,SAO PAULO,PAULISTA,PARCIAL,PRESENCIAL,DIREITO,MATUTINO,089.XXX.XXX-40,F,Branca,04/04/2003,N,NORDESTE,BA,IBITITA
4,2020,203,UNIVERSIDADE SÃO JUDAS TADEU,SAO PAULO,PAULISTA,INTEGRAL,PRESENCIAL,DIREITO,MATUTINO,173.XXX.XXX-09,F,Branca,07/12/1977,N,SUDESTE,SP,SAO PAULO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166825,2020,1491,CENTRO UNIVERSITÁRIO INTERNACIONAL,CABO FRIO,PAP CABO FRIO,PARCIAL,EAD,ADMINISTRAÇÃO,CURSO A DISTÂNCIA,062.XXX.XXX-70,F,Parda,30/11/1996,N,SUDESTE,RJ,SAO PEDRO DA ALDEIA
166826,2020,1491,CENTRO UNIVERSITÁRIO INTERNACIONAL,CABO FRIO,PAP CABO FRIO,PARCIAL,EAD,RELAÇÕES INTERNACIONAIS,CURSO A DISTÂNCIA,177.XXX.XXX-80,F,Parda,25/03/2002,N,SUDESTE,RJ,ARARUAMA
166827,2020,1491,CENTRO UNIVERSITÁRIO INTERNACIONAL,CANOAS,PAP CANOAS,PARCIAL,EAD,ADMINISTRAÇÃO,CURSO A DISTÂNCIA,042.XXX.XXX-54,M,Branca,19/04/1997,N,SUL,RS,CANOAS
166828,2020,1491,CENTRO UNIVERSITÁRIO INTERNACIONAL,ANANINDEUA,PAP ANANINDEUA,PARCIAL,EAD,RELAÇÕES INTERNACIONAIS,CURSO A DISTÂNCIA,601.XXX.XXX-53,F,Parda,17/09/1975,N,NORTE,PA,ANANINDEUA


In [ ]:
ibge = IBGE()
ibge.download(url_ibge, data_dir_ibge, start_year = 2022)


                                                 url       arquivo   ano
0  https://geoftp.ibge.gov.br/organizacao_do_terr...  DTB_2022.zip  2022
{'url': 'https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/2022/DTB_2022.zip', 'arquivo': 'DTB_2022.zip', 'ano': 2022}
Uncompress ZIP file
Remove ZIP file


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [ ]:

files = glob(f"{data_dir_ibge}*/*MUNICIPIO.xls")
for file in files:
  table_ibge = read_excel(file, skiprows=6, engine="xlrd")

map = {
    'UF': 'cod_uf',
    'Nome_UF': 'nome_uf',
    'Região Geográfica Intermediária': 'regiao_intermediaria',
    'Nome Região Geográfica Intermediária': 'nome_regiao_intermediaria',
    'Região Geográfica Imediata': 'regiao_imediata',
    'Nome Região Geográfica Imediata': 'nome_regiao_imediata',
    'Mesorregião Geográfica': 'mesorregiao_geografica',
    'Nome_Mesorregião': 'nome_mesorregiao',
    'Microrregião Geográfica': 'microrregiao_geografica',
    'Nome_Microrregião': 'nome_microrregiao',
    'Município': 'municipio',
    'Código Município Completo': 'cod_mundv',
    'Nome_Município': 'nome_municipio'
}
ufs = {
    'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA', 'Ceará': 'CE',
    'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO', 'Maranhão': 'MA', 'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG', 'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR',
    'Pernambuco': 'PE', 'Piauí': 'PI', 'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP', 'Sergipe': 'SE', 'Tocantins': 'TO'
}

regioes = {
    1: 'Norte', 2: 'Nordeste', 3: 'Sudeste', 4: 'Sul', 5: 'Centro-Oeste'
}

# Renomeando colunas e aplicando transformações
table_ibge = (
    table_ibge.rename(columns=map)
    .assign(
        sg_uf=lambda df: df['nome_uf'].map(ufs),
        cod_regiao=lambda df: df['cod_uf'].astype(str).str[1:].astype(int),
        nome_regiao=lambda df: df['cod_regiao'].map(regioes),
        nome_municipio=lambda df: df['nome_municipio'].str.upper().map(unidecode)
    )
    [['cod_mundv', 'nome_municipio', 'cod_uf', 'sg_uf', 'cod_regiao', 'nome_regiao']]
)

# Convertendo diretamente para maiúsculas (sem usar apply)
table_ibge['nome_regiao'] = table_ibge['nome_regiao'].str.upper()


table_ibge

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


,cod_mundv,nome_municipio,cod_uf,sg_uf,cod_regiao,nome_regiao
0,1100015,ALTA FLORESTA D'OESTE,11,RO,1,NORTE
1,1100379,ALTO ALEGRE DOS PARECIS,11,RO,1,NORTE
2,1100403,ALTO PARAISO,11,RO,1,NORTE
3,1100346,ALVORADA D'OESTE,11,RO,1,NORTE
4,1100023,ARIQUEMES,11,RO,1,NORTE
...,...,...,...,...,...,...
5565,5222005,VIANOPOLIS,52,GO,2,NORDESTE
5566,5222054,VICENTINOPOLIS,52,GO,2,NORDESTE
5567,5222203,VILA BOA,52,GO,2,NORDESTE
5568,5222302,VILA PROPICIO,52,GO,2,NORDESTE


In [ ]:
instituicao = prouni[['CODIGO_EMEC_IES_BOLSA','NOME_IES_BOLSA']].drop_duplicates()

map_campus = prouni['CAMPUS'].unique()
dict_campus = {key: value + 1 for value, key in enumerate(map_campus)}
prouni['cod_campus'] = prouni['CAMPUS'].map(dict_campus)
campus = prouni.copy().merge(table_ibge,
                             left_on = ['MUNICIPIO', 'REGIAO_BENEFICIARIO'],
                             right_on= ['nome_municipio', 'nome_regiao']
                             )[['cod_campus', 'cod_mundv','CAMPUS', 'CODIGO_EMEC_IES_BOLSA']]

map_turno = prouni['NOME_TURNO_CURSO_BOLSA'].unique()
dict_turno = {key: value + 1 for value, key in enumerate(map_turno)}
prouni['cod_turno'] = prouni['NOME_TURNO_CURSO_BOLSA'].map(dict_turno)
turno = prouni[['cod_turno', 'NOME_TURNO_CURSO_BOLSA']]

map_curso = prouni['NOME_CURSO_BOLSA'].unique()
dict_curso = {key: value + 1 for value, key in enumerate(map_curso)}
prouni['cod_curso'] = prouni['NOME_CURSO_BOLSA'].map(dict_curso)
curso = prouni[['cod_curso', 'NOME_CURSO_BOLSA']].drop_duplicates()

map_modalidade = prouni['MODALIDADE_ENSINO_BOLSA'].unique()
dict_modalidade = {key: value + 1 for value, key in enumerate(map_modalidade)}
prouni['cod_modalidade'] = prouni['MODALIDADE_ENSINO_BOLSA'].map(dict_modalidade)
modalidade = prouni[['cod_modalidade', 'MODALIDADE_ENSINO_BOLSA']].drop_duplicates()

map_bolsa = prouni['TIPO_BOLSA'].unique()
dict_bolsa = {key: value + 1 for value, key in enumerate(map_bolsa)}
prouni['cod_tipo_bolsa'] = prouni['TIPO_BOLSA'].map(dict_bolsa)
bolsa = prouni[['cod_tipo_bolsa', 'TIPO_BOLSA']].drop_duplicates()

prouni['cod_beneficiario'] = range(1, len(prouni) + 1)
bolsa = prouni.merge(table_ibge,
                             left_on = ['MUNICIPIO', 'UF_BENEFICIARIO'],
                             right_on=['nome_municipio', 'sg_uf']
                             )[['cod_beneficiario', 'cod_mundv', 'cod_tipo_bolsa', 'cod_curso',
               'cod_modalidade', 'cod_campus', 'CPF_BENEFICIARIO', 'SEXO_BENEFICIARIO',
               'RACA_BENEFICIARIO', 'DATA_NASCIMENTO', 'BENEFICIARIO_DEFICIENTE_FISICO' ]]

uf = table_ibge[['cod_uf', 'sg_uf']].drop_duplicates()
regiao = table_ibge[['cod_regiao', 'nome_regiao']].drop_duplicates()
municipio = table_ibge[['cod_mundv', 'nome_municipio']].drop_duplicates()



